In [1]:
# Importing necessary libraries
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# SCRAPPER

def scrapper(url):
    # making the request and getting the sites information
    r = requests.get(url)

    # getting the page html content
    webpage = bs(r.content)


    info_box =webpage.find("div", id="left-block")


    children_details = info_box.findChildren("div" , recursive=False)


    info_dict = {}
    for index,detail in enumerate(children_details): #info_box.find_all('li')
        #print(index)
        #print(children)
        #print(detail.get_text())

        # the index 0 and 2 doesn't have the information we want
        if index ==1 or index == 0 or index == 2:
            #print(index)
            #print(detail)
            #print(detail.get_text())
            #print(detail.get_text().splitlines())
            if(len(detail.get_text().splitlines()) == 3):
                info_dict['price'] = detail.get_text().splitlines()[2]

        # the index 1 have the information of price
        #elif index ==1:
            #print(detail.get_text())
            #print(url)
            
            #info_dict['price'] = detail.get_text().splitlines()[2]

        # other index have rest of the required information about the vehicle
        else:
            for data in detail.find_all('li'):
                #print(data.get_text())

                # for details having ':' in between
                if ':' in data.get_text():
                    detail = data.get_text()
                    if '\n' in data.get_text():
                        detail = detail.replace('\n', '').strip()
                        
                    info_dict[detail.split(':')[0]] = detail.split(':')[1].strip()

                # there empty 'li'  in the detail section. so to fiter it out, we use this elif condition
                elif '   ' in data.get_text():

                    # the whole text have many white spaces at beginning, so removed by .strip()
                    # in these section data_title and value are in different lines, so we use .splitlines(), 
                    detail = data.get_text().strip().splitlines()
                    #print((detail))
                    #print(detail[1])

                    # the value section have many white spaces at beginning, so removed by .strip()
                    info_dict[detail[0]] = detail[1].strip() 
                    #info_dict[data.get_text().splitlines()[1]] = info_dict[data.get_text().splitlines()[1]]
                    
                    
    return info_dict



In [28]:
# LINK GENERATOR


#Bajaj Bike: 'http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,6/iPage,1'
# Honda Bike: 'http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,12/iPage,1'
# Suzuki Bike: 'http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,19/iPage,1'
#TVS Bike: 'http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,20/iPage,1'
#Royal_Enfield Bike: 'http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,17/iPage,1'
#KTM Bike: 'http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,1/iPage,1'
#Yamaha Bike: 'http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,22/iPage,1'
#Hero Bike: 'http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,11/iPage,1'

bikes_info = []

# linca ---> linker and scrapper
def linca(url):
    # making the request and getting the sites information
    r = requests.get(url)

    # getting the page html content
    webpage = bs(r.content)
    bikes = webpage.find_all("tr",class_=["odd","even"])
    #print((bikes))
    for bike in bikes:
        #pass
        bike_link = bike.a['href']
        #print(bike_link)
        bike_info = scrapper(bike_link)
        bikes_info.append(bike_info)
        

for i in range(1,15):
    linca('http://www.dealgara.com/search/category,33/bSold,1/item_condition,All/bikeMake,11/iPage,'+str(i))
    

In [29]:
(bikes_info[0:2])

[{'price': 'Rs. 40,000',
  'Condition ': 'Used',
  'Date ': '14/7/2020',
  'Views ': '178',
  'City': 'Kathmandu',
  'City Area': 'Lazimpat',
  'Address': 'Lazimpat',
  'Motorbike Type ': 'Scooter',
  'Make Year': '2009',
  'Make': 'Hero',
  'Model': 'Pleasure',
  'Transmission': 'Manual',
  'Engine size (cc)': '102',
  'Colour': 'Black',
  'Lot No.': '38',
  'Kilometers': '32000'},
 {'price': 'Rs. 1,00,000',
  'Condition ': 'Used',
  'Date ': '19/3/2017',
  'Views ': '98',
  'City': '',
  'City Area': '',
  'Address': '',
  'Motorbike Type ': 'Standard',
  'Make Year': '2012',
  'Make': 'Hero',
  'Model': 'CBZ Xtreme',
  'Transmission': 'Manual',
  'Engine size (cc)': '149',
  'Colour': 'Black',
  'Lot No.': '4',
  'Kilometers': '60000'}]

In [30]:
len(bikes_info)

209

In [31]:
# Function: saving the above dictionary into json

import json

def save_data(title,data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
        
# Saving the scrapped bike data

save_data("Data/hero_bikes.json", bikes_info) # python dictionary works best with json